In [14]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from collections import defaultdict
import re
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

from parse_entry import EntryParser

In [15]:
# Load data
df = pd.read_csv('./data/people.csv')

In [16]:
# Create graph

def create_edges(j, df):
    
    edges = []
    
    person = df.iloc[j]
    name = person.name_en
    counts = EntryParser().edges_to(person.url)
    for v in counts.keys():
        ind = df[df['name_en'] == v].index
        if len(ind) != 0:
            i = ind[0]
            edges.append((j, i, counts[v]))
            
    return edges

results = []

p = Pool(12)
for j in range(len(df)): 
    edges = p.apply_async(create_edges, args=(j,df))
    results.append(edges)
p.close()
p.join()
print('All subprocesses done.')

graph = []
for result in results: graph.extend(result.get())

All subprocesses done.


In [17]:
# Save graph
header = ['p1', 'p2', 'count']
df_graph = pd.DataFrame(columns=header, data=graph)
df_graph.to_csv('./data/graph.csv', index = False)